In [29]:
import pandas as pd
import sys
import os

module_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [30]:
from ai_service import label_dataset_ner, AiClient
ai_service = AiClient()

In [31]:
train = pd.read_csv('../data/conll003-englishversion/train/train.csv')

delete where there are no tags

In [32]:
train = train[train['entities'] != '[]'].copy()
train.head()

,Unnamed: 0.1,Unnamed: 0,sentence_id,sentence_text,ner_tags_str,entities
0,0,0,1,EU rejects German call to boycott British lamb .,B-ORG O B-MISC O O O B-MISC O O,"[{'entity': 'EU', 'label': 'ORG'}, {'entity': ..."
1,1,1,2,Peter Blackburn,B-PER I-PER,"[{'entity': 'Peter Blackburn', 'label': 'PER'}]"
2,2,2,3,BRUSSELS 1996-08-22,B-LOC O,"[{'entity': 'BRUSSELS', 'label': 'LOC'}]"
3,3,3,4,The European Commission said on Thursday it di...,O B-ORG I-ORG O O O O O O B-MISC O O O O O B-M...,"[{'entity': 'European Commission', 'label': 'O..."
4,4,4,5,Germany 's representative to the European Unio...,B-LOC O O O O B-ORG I-ORG O O O B-PER I-PER O ...,"[{'entity': 'Germany', 'label': 'LOC'}, {'enti..."


In [33]:
train = train.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11132 entries, 0 to 14040
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   sentence_id    11132 non-null  int64 
 1   sentence_text  11132 non-null  object
 2   ner_tags_str   11132 non-null  object
 3   entities       11132 non-null  object
dtypes: int64(1), object(3)
memory usage: 434.8+ KB


In [34]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11132 entries, 0 to 14040
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   sentence_id    11132 non-null  int64 
 1   sentence_text  11132 non-null  object
 2   ner_tags_str   11132 non-null  object
 3   entities       11132 non-null  object
dtypes: int64(1), object(3)
memory usage: 434.8+ KB


In [35]:
tamplate = {
    "role": "system",
    "content": """
        You are an expert Named Entity Recognition (NER) model. 
        Your task is to extract entities with maximum precision and return only valid JSON. 
        **Never output anything except JSON.**
        
        Rules:
            - Labels allowed: PER (Person), ORG (Organization), LOC (Location), MISC (Miscellaneous).
            - If no entities found, return {{'entities': []}}
            
        JSON Structure Example:
        {{
          entities: [
            {{
              entity: [Example Entity Name],\n"
              label: PER|ORG|LOC|MISC \n"
            }}
          ]
        }}
        Text:\n{text}
    """
}

In [36]:
sample_df = train.sample(n=1300, random_state=42).reset_index(drop=True)

In [44]:
sample_df.head(5)
sample_df.to_csv('human_train.csv')

# TEST

In [41]:
sample_indices = train.sample(n=1300, random_state=42).index
test_df = train.drop(sample_indices).reset_index(drop=True)

display(test_df.head(2))
display(test_df.info())

test_df.to_csv('../data/conll003-englishversion/test/test.csv')

,sentence_id,sentence_text,ner_tags_str,entities
0,1,EU rejects German call to boycott British lamb .,B-ORG O B-MISC O O O B-MISC O O,"[{'entity': 'EU', 'label': 'ORG'}, {'entity': ..."
1,2,Peter Blackburn,B-PER I-PER,"[{'entity': 'Peter Blackburn', 'label': 'PER'}]"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9832 entries, 0 to 9831
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   sentence_id    9832 non-null   int64 
 1   sentence_text  9832 non-null   object
 2   ner_tags_str   9832 non-null   object
 3   entities       9832 non-null   object
dtypes: int64(1), object(3)
memory usage: 307.4+ KB


None

In [39]:
res_df = label_dataset_ner(sample_df, tamplate, ai_service.gemini_call, model_name='gemini-2.0-flash')
res_df.to_csv("ner_train_gemini.csv")

  0%|          | 0/1300 [00:00<?, ?it/s]

100%|██████████| 1300/1300 [16:54<00:00,  1.28it/s]


In [40]:
res_df.head()

,sentence_id,sentence_text,ner_tags_str,entities,entities_pred,llm_time,llm_prompt_tokens,llm_completion_tokens
0,7567,Egyptian government newspapers have criticised...,B-MISC O O O O B-LOC O O B-MISC O O O O O O O ...,"[{'entity': 'Egyptian', 'label': 'MISC'}, {'en...","[{'entity': 'Egyptian government', 'label': 'O...",0.844970,203,62
1,624,Coritiba 1 Atletico Mineiro 0,B-ORG O B-ORG I-ORG O,"[{'entity': 'Coritiba', 'label': 'ORG'}, {'ent...","[{'entity': 'Coritiba', 'label': 'ORG'}, {'ent...",0.696006,171,66
2,12278,"4. Greece ( Dimitrios Georgalis , Georgios Chi...",O B-LOC O B-PER I-PER O B-PER I-PER O B-PER,"[{'entity': 'Greece', 'label': 'LOC'}, {'entit...","[{'entity': 'Greece', 'label': 'LOC'}, {'entit...",0.867434,178,114
3,10981,"Chile 's ENAP buys Oriente , Escravos crude fo...",B-LOC O B-ORG O B-ORG O B-ORG O O O O,"[{'entity': 'Chile', 'label': 'LOC'}, {'entity...","[{'entity': 'Chile', 'label': 'LOC'}, {'entity...",1.063560,175,109
4,6961,"The Swiss teenager , a twice French Open junio...",O B-MISC O O O O B-MISC I-MISC O O O O B-MISC ...,"[{'entity': 'Swiss', 'label': 'MISC'}, {'entit...","[{'entity': 'Swiss', 'label': 'MISC'}, {'entit...",0.944853,196,108
